In [35]:
import pandas as pd
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from autorag.data.qacreation.ragas import generate_qa_ragas

import getpass
import os


Imported libraries:
builtins
dotenv
getpass
importlib
json
markdown
os
pandas
re
sys
types


In [28]:
from dotenv import load_dotenv
load_dotenv()

if "HF_API_KEY" not in os.environ:
    os.environ["HF_API_KEY"] = getpass("Enter HF API key:")

In [29]:
corpus_df = pd.read_parquet("../autorag/data/corpus.parquet", engine='pyarrow')


In [30]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-70B",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

[07/03/24 14:15:20] INFO     [SentenceTransformer.py:189] >> Use pytorch device_name:    ]8;id=130725;file:///Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=2973;file:///Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/sentence_transformers/SentenceTransformer.py#189\189]8;;\
                             mps                                                                                   

                    INFO     [SentenceTransformer.py:197] >> Load pretrained             ]8;id=988561;file:///Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=333831;file:///Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/sentence_transformers/SentenceTransformer.py#197\197]8;;\
                             SentenceTransformer:                                                                  
                             sentence-transformers/all-mpnet-base-v2                                               

In [21]:
distributions = {  # uniform distribution
    simple: 0.1,
    reasoning: 0.35,
    multi_context: 0.2,
    conditional: 0.35
}
qa_df = generate_qa_ragas(corpus_df, test_size=50, 
                          generator_llm=llm, 
                          critic_llm=llm, 
                          embedding_model=embeddings, 
                          distributions=distributions)

embedding nodes:   0%|          | 0/15414 [00:00<?, ?it/s]

KeyboardInterrupt: 

Exception ignored in: Exception ignored in: <generator object tqdm.__iter__ at 0x31c524ca0>
Traceback (most recent call last):
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/notebook.py", line 178, in display
    pbar.bar_style = bar_style
    ^^^^^^^^^^^^^^
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/traitlets/traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/traitlets/traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packa

In [17]:
qa_df.to_parquet('../data/evaluation/ragas_llama3_qa.parquet')